# Title

#### Subtitle

In [15]:
# imports

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## Summary

## Introduction

## Methods

### Data

### Analysis

## Results & Discussion

In [13]:
# import raw data

colnames = ['class','buying','maint','doors','persons','lug_boot','safety']
car_data = pd.read_csv('../data/raw/car.data', names=colnames, header=None)
# car_data.isnull().any() # categorigal features, no missing values

car_data

,class,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [17]:
# train test split, export to csv
np.random.seed(522)

car_train, car_test = train_test_split(car_data, train_size = 0.8, stratify=car_data['class'])
car_train.to_csv('../data/processed/car_train.csv')
car_test.to_csv('../data/processed/car_test.csv')

In [ ]:
# preprocessing

car_preprocessor = make_column_transformer()

# ... some EDA and plots

## References